In [1]:
import numpy  as np
import pandas as pd
from metrics import MAE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../../../Datasets/final/X_train.pkl")
c = X_train.columns.tolist()
X_val = pd.read_pickle("../../../Datasets/final/X_val.pkl")
X_test = pd.read_pickle("../../../Datasets/final/X_test.pkl")
y_train = pd.read_pickle("../../../Datasets/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../Datasets/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../Datasets/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
state_columns = [i for i in c if "state" in i]

In [5]:
mean_per_state_dict = {}
for state in state_columns:
    state_df = X_train.groupby([state, "gPlusPlaceId"]).agg({"gPlusUserId": "count"}).reset_index()
    popular_place = state_df[state_df[state] == 1].sort_values("gPlusUserId", ascending=False).iloc[0]["gPlusPlaceId"]
    mean_per_state_dict[state] = X_train[(X_train[state] == 1) & (X_train["gPlusPlaceId"] == popular_place)]["rating"].mean()

In [6]:
mean_per_state_dict

{'state_AL': 4.16,
 'state_AR': 3.9285714285714284,
 'state_AZ': 1.7098765432098766,
 'state_CA': 3.611764705882353,
 'state_CO': 3.0142857142857142,
 'state_CT': 4.035714285714286,
 'state_DE': 3.5294117647058822,
 'state_GA': 1.2307692307692308,
 'state_IA': 4.36,
 'state_ID': 4.391304347826087,
 'state_IL': 4.589743589743589,
 'state_IN': 4.463636363636364,
 'state_KS': 4.545454545454546,
 'state_KY': 4.619047619047619,
 'state_LA': 3.5555555555555554,
 'state_MA': 4.264150943396227,
 'state_MD': 4.891304347826087,
 'state_ME': 4.777777777777778,
 'state_MI': 4.653846153846154,
 'state_MN': 4.586956521739131,
 'state_MO': 4.5636363636363635,
 'state_MS': 4.25,
 'state_MT': 4.357142857142857,
 'state_NC': 4.018518518518518,
 'state_ND': 3.25,
 'state_NE': 4.1,
 'state_NH': 3.4545454545454546,
 'state_NJ': 3.868131868131868,
 'state_NM': 3.68,
 'state_NV': 4.928571428571429,
 'state_NY': 4.608695652173913,
 'state_OH': 4.379310344827586,
 'state_OK': 4.366666666666666,
 'state_OR': 4.

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][state_columns])):
        if X_val.iloc[i][state_columns][j] == 1:
            y_pred_val.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [08:54<00:00, 302.24it/s]


In [8]:
MAE(predictions=y_pred_val, labels=y_val)[0]

1.1879910578491157

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][state_columns])):
        if X_test.iloc[i][state_columns][j] == 1:
            y_pred.append(mean_per_state_dict[state_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [08:59<00:00, 299.63it/s]


In [10]:
MAE(predictions=y_pred, labels=y_test)[0]

1.180809783216159